In [1]:
%autosave 0
import requests
import io
import pandas as pd
from datetime import date, timedelta

## scrap data from source
url="https://covid.ourworldindata.org/data/owid-covid-data.csv"
s=requests.get(url).content
c=pd.read_csv(io.StringIO(s.decode('utf-8')))

## this dataset sometime no receive update, so i check the simple link 
verif_data = c[c['date'].isin([f"{date.today()}"])]

if len(verif_data) == 0:
    print("Not today, check other link")
    url="https://covid.ourworldindata.org/data/ecdc/full_data.csv"
    s=requests.get(url).content
    c=pd.read_csv(io.StringIO(s.decode('utf-8')))
    
    verif_data = c[c['date'].isin([f"{date.today()}"])]
    
    ## If with the second link data have not yet be updated, raise Error & try again later
    if len(verif_data) == 0:
        verif_data = c[c['date'].isin([f"{date.today()}"])]
        raise ValueError('Data have not receive Update at this moment, Try later !')
    
verif_data.head(3)

Autosave disabled


,iso_code,continent,location,date,total_cases,new_cases,new_cases_smoothed,total_deaths,new_deaths,new_deaths_smoothed,...,aged_70_older,gdp_per_capita,extreme_poverty,cardiovasc_death_rate,diabetes_prevalence,female_smokers,male_smokers,handwashing_facilities,hospital_beds_per_thousand,life_expectancy
240,AFG,Asia,Afghanistan,2020-08-27,38126.0,55.0,52.429,1401.0,4.0,2.571,...,1.337,1803.987,NaN,597.029,9.59,NaN,NaN,37.746,0.50,64.83
412,ALB,Europe,Albania,2020-08-27,8927.0,168.0,159.286,263.0,4.0,4.143,...,8.643,11803.431,1.1,304.195,10.08,7.1,51.2,NaN,2.89,78.57
653,DZA,Africa,Algeria,2020-08-27,42619.0,391.0,396.000,1465.0,9.0,9.000,...,3.857,13913.839,0.5,278.364,6.73,0.7,30.4,83.741,1.90,76.88


In [2]:
## dump the file for using it in other nb
c.to_csv(f"AzureSC\\Base_Files\\full_data{date.today()}.csv", index=False)

In [3]:
## Load data from days -1 to evaluate the performances of model
data_load = pd.read_csv(f"AzureSC\\Pred\\predDf{date.today() - timedelta(days=1)}.csv", index_col=0)
data_load

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-08-26,France,249903.0,0.0,30578.0,0.0,0.0,0.0
0,2020-08-26,China,89785.0,0.0,4718.0,0.0,0.0,0.0
0,2020-08-26,Italy,262083.0,0.0,35470.0,0.0,0.0,0.0
0,2020-08-26,Spain,426586.0,0.0,28962.0,0.0,0.0,0.0
0,2020-08-26,United States,5867361.0,0.0,179277.0,0.0,0.0,0.0
0,2020-08-26,World,24459854.0,0.0,827361.0,0.0,0.0,0.0
0,2020-08-26,United Kingdom,328915.0,0.0,41489.0,0.0,0.0,0.0
0,2020-08-26,Germany,237586.0,0.0,9290.0,0.0,0.0,0.0
0,2020-08-26,Iran,365398.0,0.0,21019.0,0.0,0.0,0.0
0,2020-08-26,Turkey,262402.0,0.0,6179.0,0.0,0.0,0.0


In [4]:
def Eval(c,data_load,country):
    """ Factorise function for evaluate all models
    params : 
        c = data scrap today
        data_load = dataset from day -1
        country = country concerned by eval
    """
    df = c
    df = df[df['location'].isin([f"{country}"])]
    sle = df.iloc[-1]
    v0 = sle["date"]
    v1 = sle["total_cases"]
    v2 = sle["total_deaths"]
    
    res_tempo = data_load[data_load['country'].isin([f"{country}"])]
    rez1 = res_tempo.loc[0,"total_cases_predict"]
    rez2 = res_tempo.loc[0,"total_deaths_predict"]
    
    errorCase1 = rez1-v1
    errorCase2 = rez2-v2

    follow_df = pd.DataFrame()
    follow_df.loc[0,"date"] = v0
    follow_df.loc[0,"country"] = country
    follow_df.loc[0,"total_cases_predict"] = rez1
    follow_df.loc[0,"total_cases_real"] = v1
    follow_df.loc[0,"total_deaths_predict"] = rez2
    follow_df.loc[0,"total_deaths_real"] = v2
    follow_df.loc[0,"error_abs_cases"] = errorCase1
    follow_df.loc[0,"error_abs_deaths"] = errorCase2
    
    return follow_df

In [5]:
## call eval function and store dataframe on var's
follow_df1 = Eval(c,data_load,"France")
follow_df2 = Eval(c,data_load,"China")
follow_df3 = Eval(c,data_load,"Italy")
follow_df4 = Eval(c,data_load,"Spain")
follow_df5 = Eval(c,data_load,"United States")
follow_df6 = Eval(c,data_load,"World")
follow_df7 = Eval(c,data_load,"United Kingdom")
follow_df8 = Eval(c,data_load,"Germany")
follow_df9 = Eval(c,data_load,"Iran")
follow_df10 = Eval(c,data_load,"Turkey")
follow_df11 = Eval(c,data_load,"Brazil")

In [6]:
## concatenate all df in 1
frames = [follow_df1, follow_df2, follow_df3, follow_df4,
          follow_df5, follow_df6, follow_df7, follow_df8,
          follow_df9, follow_df10, follow_df11]
rapport = pd.concat(frames)
rapport

,date,country,total_cases_predict,total_cases_real,total_deaths_predict,total_deaths_real,error_abs_cases,error_abs_deaths
0,2020-08-27,France,249903.0,253587.0,30578.0,30544.0,-3684.0,34.0
0,2020-08-27,China,89785.0,89784.0,4718.0,4713.0,1.0,5.0
0,2020-08-27,Italy,262083.0,262540.0,35470.0,35458.0,-457.0,12.0
0,2020-08-26,Spain,426586.0,419849.0,28962.0,28971.0,6737.0,-9.0
0,2020-08-27,United States,5867361.0,5821876.0,179277.0,179714.0,45485.0,-437.0
0,2020-08-27,World,24459854.0,24204686.0,827361.0,826090.0,255168.0,1271.0
0,2020-08-27,United Kingdom,328915.0,328846.0,41489.0,41465.0,69.0,24.0
0,2020-08-27,Germany,237586.0,237936.0,9290.0,9285.0,-350.0,5.0
0,2020-08-27,Iran,365398.0,363363.0,21019.0,20901.0,2035.0,118.0
0,2020-08-27,Turkey,262402.0,262507.0,6179.0,6183.0,-105.0,-4.0


In [7]:
## dump dataset for use it in nb 03, in 03 nb he will be add to full_rapport then delete 
rapport.to_csv(f"AzureSC\\Rapport\\rap{date.today()}.csv")